In [3]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix 
path_build = "../../data/tencent2020/build2/"
path_embed = "../../data/tencent2020/embed2/"

path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle"
sub_path = "../../data/tencent2020/sub"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)

In [3]:
user_ids = pd.read_pickle(f"{path_build}/user_ids.pkl")
#user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
#user_ids = user_ids.merge(user, how='left', on='user_id')
user_ids.head()

,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,ad_id_times,product_id_times,advertiser_id_times,product_category_times,industry_times
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[10005, 136, 10005, 10005, 1334, 10018, 10005,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[326, 6, 106, 326, 100, 326, 73, 217, 64, 238,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[1063182, 293331, 1138617, 2130126, 176368, 10...","[958580, 269566, 1026121, 1889685, 162737, 997...","[20301, 490, 20301, 20301, 2679, 20352, 20301,...","[12541, 16889, 58163, 26349, 20377, 26387, 315...","[30, 7, 30, 30, 7, 108, 30, 30, 108, 7, 7, 7, 8]","[1732, 39, 526, 1732, 499, 1732, 380, 1102, 34..."
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...","[58788, 139702, 38066, 541125, 14495, 392680, ...","[87, 80, 129, 129, 1400, 87, 10018, 1261, 1001...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[22885, 10686, 18562, 25932, 768, 22885, 34505...","[318, 238, 6, 6, 317, 318, 47, 6, 47, 47, 242,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[93754, 222023, 58596, 790086, 18933, 581326, ...","[88571, 203960, 56789, 716490, 20602, 528123, ...","[277, 238, 459, 459, 2899, 277, 20352, 2444, 2...","[44043, 18905, 34552, 50889, 1333, 44043, 6847...","[7, 7, 7, 7, 7, 7, 108, 7, 108, 108, 7, 108, 7...","[1679, 1246, 39, 39, 1671, 1679, 262, 39, 262,..."
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...","[586668, 713448, 629278, 728308, 527601, 64310...","[36256, 40905, 1674, 35985, 1674, 10018, 10018...","[17, 17, 2, 17, 2, 18, 18, 2, 2, 2, 18, 12, 2,...","[32974, 9877, 18492, 14186, 17018, 9058, 8371,...","[100, 100, 322, 100, 322, 6, 54, 6, 322, 322, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12, 13, 14, 14, 14, 17, 19, 22, 31, 36, 37, 4...","[855771, 1045997, 918547, 1068492, 770408, 939...","[774925, 943201, 830445, 963534, 698956, 84889...","[59449, 64457, 3722, 59160, 3722, 20352, 20352...","[65335, 17287, 34379, 25498, 31243, 15690, 145...","[102, 102, 7, 102, 7, 108, 108, 7, 7, 7, 108, ...","[499, 499, 1711, 499, 1711, 39, 292, 39, 1711,..."
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...","[37966, 524312, 511235, 1638619, 1698206, 1953...","[1862, 10018, 2625, 38743, 41265, 37758, 39904...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[19451, 7976, 13084, 12130, 23664, 811, 10172,...","[238, 25, 248, 100, 100, 100, 100, 100, 88, 31...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[8, 15, 41, 44, 48, 48, 48, 48, 49, 52, 58, 58...","[58389, 765852, 747484, 2384416, 2469119, 2836...","[56622, 694770, 678514, 2113043, 2186873, 2505...","[4294, 20352, 7118, 62143, 64855, 61056, 63389...","[36447, 13800, 23470, 21647, 45737, 1414, 1784...","[7, 108, 7, 102, 102, 23, 102, 102, 108, 7, 7,...","[1246, 136, 1308, 499, 499, 499, 499, 499, 452..."
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...","[265971, 314795, 24966, 41148, 751113, 925792,...","[10005, 10008, 87, 136, 10018, 136, 10005, 100...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[11882, 992, 22885, 9706, 38760, 2862, 17745, ...","[297, 100, 318, 6, 322, 6, 288, 322, 319, 238,...","[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 13, 14, 15, 20, 21, 24, 25, 27, 28, 29, 30...","[401225, 469363, 35978, 63924, 1105070, 135513...","[367911, 429875, 36734, 61628, 995584, 1216710...","[20301, 20316, 277, 490, 20352, 491, 20301, 20...","[21154, 1925, 44043, 16910, 76947, 5468, 32796...","[30, 50, 7, 7, 108, 8, 30, 108, 108, 7, 108, 7...","[1562, 499, 1679, 39, 1711, 40, 1494, 1711, 16..."


In [4]:
w2v_features = [
    {'name':'creative_id', 'size':256, 'windows':10, 'min_count':1, 'version':1},
    {'name':'ad_id', 'size':256, 'windows':10, 'min_count':1, 'version':1},
    {'name':'advertiser_id', 'size':64, 'windows':10, 'min_count':1, 'version':1},
    {'name':'product_id', 'size':64, 'windows':10, 'min_count':1, 'version':1},
    {'name':'industry', 'size':32, 'windows':10, 'min_count':1, 'version':1},
    {'name':'product_category', 'size':16, 'windows':10, 'min_count':1, 'version':1},
    {'name':'time', 'size':16, 'windows':10, 'min_count':1, 'version':1},
    {'name':'click_times', 'size':8, 'windows':10, 'min_count':1, 'version':1},
    
    {'name':'creative_id_times', 'size':256, 'windows':10, 'min_count':1, 'version':1},
    {'name':'ad_id_times', 'size':256, 'windows':10, 'min_count':1, 'version':1},
    {'name':'product_id_times', 'size':64, 'windows':10, 'min_count':1, 'version':1},
    {'name':'advertiser_id_times', 'size':64, 'windows':10, 'min_count':1, 'version':1},
    {'name':'product_category_times', 'size':32, 'windows':10, 'min_count':1, 'version':1},
    {'name':'industry_times', 'size':32, 'windows':10, 'min_count':1, 'version':1},
    
##     {'name':'creative_id_t', 'size':128, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'ad_id_t', 'size':128, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'product_id_t', 'size':64, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'advertiser_id_t', 'size':64, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'product_category_t', 'size':32, 'windows':5, 'min_count':1, 'version':2},
##     {'name':'industry_t', 'size':32, 'windows':5, 'min_count':1, 'version':2},
    
#     {'name':'time_clicktimes', 'size':91, 'windows':10, 'min_count':1, 'version':2,},
#     {'name':'time_creativeids', 'size':91, 'windows':10, 'min_count':1, 'version':2 },
    
]
for fea_dict in w2v_features:
    print(fea_dict)
    name = fea_dict['name']
    size = fea_dict['size']
    windows = fea_dict['windows']
    min_count = fea_dict['min_count']
    version = fea_dict['version']
    user_ids[name] = user_ids[name].map(lambda x: [str(i) for i in x])
    w2v = models.Word2Vec(user_ids[name].values, sg=1, size=size, window=windows, workers=6, hs=1, min_count=min_count)
    w2v.wv.save_word2vec_format(f"{path_embed}/{name}_w2v_{windows}_{version}.bin",binary=True)
    #保存矩阵
    w2v = models.KeyedVectors.load_word2vec_format(f"{path_embed}/{name}_w2v_{windows}_{version}.bin",binary=True)
    if 'vocab_size' in fea_dict:
        w2v_vocab_size = fea_dict['vocab_size']
    else:
        w2v_vocab_size = len(w2v.vocab) + 5
        print(w2v_vocab_size)
    w2v_matrix = np.zeros((w2v_vocab_size, w2v.vector_size))
    for w in w2v.vocab:
        w2v_matrix[int(w)] = w2v[w]
    save_pickle(w2v_matrix.astype(np.float16), f"{path_embed}/{name}_w2v_matrix.pkl",)

{'name': 'creative_id', 'size': 256, 'windows': 10, 'min_count': 1, 'version': 1}
4445722
